## Problem Statement:

# "Predicting the price of Wine with the Keras Functional API and Tensorflow"

- Building a wide and deep network using Keras (tf.Keras) to predict the price of wine from its description.
- Dataset available at https://github.com/EshitaNandy/Prediction-using-Keras-and-Tensorflow/

- The overall goal is to create a model that can identify the variety, winery and location of a wine based on a description.

- Prerequisites : Jupyter Notebook, Pandas, Numpy, Scikitlearn and Keras (Tensorflow)

## Here are all the imports that we will require to build this model.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import itertools
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
layers = keras.layers

In [3]:
## This code is for testing purpose of proper Tensorflow Execution
print("You have an IT job", tf.__version__)

You have an IT job 2.1.0


In [4]:
data = pd.read_csv(r"C:\Users\User\Downloads\KERAS PYTHON\winemag-data-130k-v2.csv")
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## Do some preprocessing to remove null values and drop unnecessary columns:

In [5]:
data = data[pd.notnull(data['country'])]
data = data[pd.notnull(data['price'])]
data = data.drop(data.columns[0], axis =1)
data.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem


## Do some preprocessing to limit wine varieties in the dataset

In [6]:
variety_threshold = 500 ## Anything that occurs less than this will be removed
value_counts = data['variety'].value_counts()
to_remove = value_counts[value_counts <= variety_threshold].index
data.replace(to_remove,np.nan, inplace=True)
data = data[pd.notnull(data['variety'])]
data.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach


## Split the Data into Training Dataset and Testing Dataset

In [7]:
train_size = int(len(data)* 0.8)
print("Train size: %d" % train_size) 
print("Train size: %d" % (len(data) - train_size))

Train size: 82431
Train size: 20608


## Training and Testing on Features and Labels

In [8]:
#Train Features
description_train = data['description'][:train_size]
variety_train = data['variety'][:train_size]

#Train Labels
labels_train = data['price'][:train_size]

#Test Features
description_test = data['description'][train_size:]
variety_test = data['variety'][train_size:]

#Test Labels
labels_test = data['price'][train_size:]

## Create a tokenizer to preprocess our text description.

In [9]:
vocab_size = 12000 # this is a hyperparameter
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(description_train) # only fit on train

## Wide Feature 1: sparse bag of words(bow) vocab_size vector

In [10]:
description_bow_train = tokenize.texts_to_matrix(description_train)
description_bow_test = tokenize.texts_to_matrix(description_test)

## Wide Feature 2: one-hot vector of variety categories

- Use sklearn utility to convert label strings to numbered index

In [11]:
import sklearn
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(variety_train)
variety_train = encoder.transform(variety_train)
variety_test = encoder.transform(variety_test)
num_classes = np.max(variety_train)+1

- Convert labels to one hot

In [12]:
variety_train = keras.utils.to_categorical(variety_train, num_classes)
variety_test = keras.utils.to_categorical(variety_test, num_classes)

## Defining our wide model with the functional API

In [13]:
bow_inputs = layers.Input(shape=(vocab_size,))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs,variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)

In [14]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(wide_model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12000)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 38)]         0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 12038)        0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          3081984     concatenate[0][0]            

## Deep Model Feature : Word Embeddings of Wine Description.

In [15]:
train_embed = tokenize.texts_to_sequences(description_train)
test_embed = tokenize.texts_to_sequences(description_test)
max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

## Define our Deep Model with the Functional API

In [16]:
deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs=deep_inputs, outputs = embed_out)
print(deep_model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 170)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 170, 8)            96000     
_________________________________________________________________
flatten (Flatten)            (None, 1360)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1361      
Total params: 97,361
Trainable params: 97,361
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
deep_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

## Combine wide and deep into one model

In [18]:
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
combined_model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
print(combined_model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12000)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 38)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 170)]        0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 12038)        0           input_1[0][0]                    
                                                                 input_2[0][0]              

## Run Training

In [19]:
# Training
combined_model.fit([description_bow_train, variety_train] + [train_embed], labels_train, epochs=10, batch_size=128)

# Evaluation
combined_model.evaluate([description_bow_test, variety_test] + [test_embed], labels_test, batch_size=128)

Train on 82431 samples
Epoch 1/10
82431/82431 [==============================] - 71s 861us/sample - loss: 1487.0239 - accuracy: 0.0000e+00
Epoch 2/10
82431/82431 [==============================] - 67s 818us/sample - loss: 1272.5073 - accuracy: 0.0000e+00
Epoch 3/10
82431/82431 [==============================] - 87s 1ms/sample - loss: 1168.8411 - accuracy: 0.0000e+00
Epoch 4/10
82431/82431 [==============================] - 76s 924us/sample - loss: 1069.7310 - accuracy: 0.0000e+00
Epoch 5/10
82431/82431 [==============================] - 62s 746us/sample - loss: 964.1559 - accuracy: 0.0000e+00
Epoch 6/10
82431/82431 [==============================] - 59s 715us/sample - loss: 855.4663 - accuracy: 0.0000e+00- loss: 827.7758 - accura
Epoch 7/10
82431/82431 [==============================] - 58s 707us/sample - loss: 748.1084 - accuracy: 0.0000e+00
Epoch 8/10
82431/82431 [==============================] - 60s 730us/sample - loss: 639.2863 - accuracy: 0.0000e+00
Epoch 9/10
82431/82431 [======

[1461.2687954162218, 0.0]

## Generating predictions on our trained model

In [20]:
predictions = combined_model.predict([description_bow_test, variety_test] + [test_embed])

## Compare predictions with actual value for the first few items in our test dataset

In [21]:
num_predictions = 40
diff = 0

for i in range(num_predictions):
    val = predictions[i]
    print(description_test.iloc[i])
    print('Predicted Price: ',val[0], 'Actual Price: ', labels_test.iloc[i], '\n')
    diff += abs(val[0] - labels_test.iloc[i])

While aromatically subdued, this is a refreshing, easy-drinking white blend, with pleasant lemon and apple flavors. A hint of orange peel and a brisk, lime-accented acidity highlight the midpalate. Drink now.
Predicted Price:  13.683121 Actual Price:  10.0 

A new product from the Tommasi family (celebrated for its Amarone and other hearty red wines), this is light and fresh, with aromas of citrus, stone fruit and white flowers.
Predicted Price:  14.357279 Actual Price:  13.0 

From Italian tenor Andrea Bocelli and his family (vintners in Tuscany for 130 years) comes this bright, cheerful sparkler. It shows crisp acidity, ease, tonic effervescence and overall harmony.
Predicted Price:  10.050094 Actual Price:  19.0 

“Bolle,” which is Italian for “bubbles,” plays up its cheerful and playful side with bright aromas of citrus, peach, passion fruit and green apple. It washes down easily thanks to the wine's natural acidity.
Predicted Price:  7.1370635 Actual Price:  15.0 

Pale red in col

## Compare the average difference between Actual price and the Model's Predicted price

In [23]:
print('Average Prediction Difference: ', diff/num_predictions)

Average Prediction Difference:  13.613453662395477


## So, the average prediction difference is around 13Dollars in every bottle of wine which is really a good value to move ahead with. 